In [188]:
def init_parameters(num_hidden_layers, activation_function, sizes, inputsize, outputsize): #here input size and output size are fixed as per problem this can be modified as per requirement

  sizes = [inputsize] + sizes

  sizes = sizes+ [outputsize]

  np.random.seed(1234)

  parameters={}

  if activation_function == "relu":
    parameters["W" + str(i)] = 0.01*np.random.randn(sizes[i], sizes[i-1])*(np.sqrt(2/(sizes[i]+sizes[i-1])))
    parameters["b" + str(i)] = np.zeros(sizes[i],1)

  else:
    for i in range(1, num_hidden_layers+2):
      parameters["W" + str(i)] = np.random.randn(sizes[i], sizes[i-1])   #initializing weight matrix
      parameters["b" + str(i)] = np.zeros(sizes[i],1)   #initializing bias vector

  return parameters

In [189]:
def fwd_prop(X, parameters, activation_func, num_hidden_layers):

    A={}
    activation = Activation(activation_func)
    if X.ndim == 1:
      X=X[:,np.newaxis]

    H = {"h0":X}

    for l in range(1,num_hidden_layers+2):
      Wl=parameters["W"+str(l)]
      bl=parameters["b"+str(l)]

      hprev = H["h" + str(l-1)]
      al = np.dot(Wl, hprev) +bl
      A["a"+str(l)] = al

      if l!= num_hidden_layers+1:
        hl = activation.activate(al)

      elif l == num_hidden_layers+1:
        hl=softmax(al)

      H["h"+str(l)] = hl


    yhat=H["h"+str(num_hidden_layers+1)]

    return yhat, A, H

In [190]:
#calculating loss
class loss_class:
    def __init__(self, loss_method):
        self.loss_method = loss_method
        
    def calc_loss(self, y_pred, y_act):
        y_pred = np.array(y_pred)
        y_act = np.array(y_act)        

        if y_pred.ndim == 1:
            y_pred = y_pred[:,np.newaxis]

        if y_act.ndim == 1:
            y_act = y_act[:, np.newaxis]
        
        if self.loss_method == 'cross_entropy':
            loss = -1*y_act*np.log(y_pred)
            loss = np.sum(loss)
            #print(loss)
        elif self.loss_method == 'squared_loss':
            loss = 0.5*np.sum((y_pred - y_act)**2)
      
        return loss


In [191]:
def softmax(a):
  z = a-np.max(a)   #check why is this necessary
  numerator = np.exp(z)
  denom = np.sum(numerator, axis = 0)

  return numerator/denom    # returns the list of the output

In [192]:
class Activation:

  def __init__(self, acti ):
    self.acti = acti



  def activate(self, a, derivative = False):
    if derivative == False:
      if self.acti == 'sigmoid':
        return 1/(1+np.exp(-a))

      elif self.acti == 'tanh':
        return np.tanh(a)

      elif self.acti == 'relu':
        return ((a>0)*(a))+((a<0)*(0.01)*a)      #leaky relu



    elif derivative == True:
      if self.acti == 'sigmoid':
        sig = 1/(1+np.exp(-a))
        return sig*(1-sig)


      elif self.acti == 'tanh':
        return (1-((np.tanh(a)**2)))

      elif self.acti == 'relu':
        return (a>0)*(np.ones(np.shape(a))) + (a<0)*(0.01*np.ones(np.shape(a)))     #leaky relu


In [193]:
def back_prop(H, A, parameters, num_hidden_layers, sizes, Y, Yhat, loss, activation_func, inputsize, outputsize):

  #gradient wrt a_L(outp-put-pre-activation)
  activation = Activation(activation_func)
  # first step in the pseudo code
  grad_one_eg = creategrads(num_hidden_layers, sizes, inputsize , outputsize)

  A["a0"] = np.zeros((inputsize,1))

  if Y.ndim == 1:
      Y = Y[:, np.newaxis]
  if Yhat.ndim == 1:
      Yhat = Yhat[:, np.newaxis]
    
  if loss == "cross_entropy" :
    #print(Yhat-Y)
    grad_one_eg["da"+str(num_hidden_layers+1)] =np.around(Yhat - Y,4)
    #print(grad_one_eg)

  elif loss == 'squared_loss':
    grad_one_eg["da" + str(num_hidden_layers+1)] = (Yhat - Y)*Yhat - Yhat*(np.dot((Yhat - Y).T, Yhat))

  #second step in psuedo code
  for i in np.arange(num_hidden_layers + 1, 0, -1):
    #gradient of weights
    grad_one_eg["dW"+str(i)] = np.dot(grad_one_eg["da"+str(i)], (H["h"+str(i-1)]).T)

    #gradient of biases
    grad_one_eg['db' + str(i)] = grad_one_eg["da" + str(i)]


    grad_one_eg["dh"+str(i-1)] = np.dot((parameters["W" + str(i)]).T , grad_one_eg["da"+str(i)])

    g_dash = activation.activate(A["a" + str(i-1)], derivative = True)

    grad_one_eg["da" + str(i-1)] = (grad_one_eg["dh" + str(i-1)])*g_dash
  #print(grad_one_eg)
  return grad_one_eg



In [194]:
def creategrads(num_hidden_layers, sizes, inputsize, outputsize):

  sizes = [inputsize] + sizes

  sizes = sizes + [outputsize]
  
  grads={"da0":np.zeros((inputsize,1)),"dho":np.zeros((inputsize,1))}

  for i in range(1, num_hidden_layers+2):
        grads["dW" + str(i)] = np.zeros((sizes[i], sizes[i-1]))
        grads["db" + str(i)] = np.zeros((sizes[i],1))
        grads["da" + str(i)] = np.zeros((sizes[i],1))
        grads["dh" + str(i)] = np.zeros((sizes[i],1))

  return grads

  #initializing all gradient matrices with zero to fill up while back propogation

In [195]:
def createnetwork(num_hidden_layers, activation_func, sizes, inputsize, outputsize):
    sizes = [inputsize]+sizes+[outputsize]
    np.random.seed(1234)
    parameters = {}
          
    if activation_func == 'relu':
        for i in range(1,num_hidden_layers+2):
            parameters['W'+str(i)] = 0.01*np.random.randn(sizes[i],sizes[i-1])*(np.sqrt(2/sizes[i]+sizes[i-1]))
            parameters['b'+str(i)] = np.zeros((sizes[i],1))
        
    else:
        for i in range(1, num_hidden_layers+2):
            parameters['W'+str(i)] = np.random.randn(sizes[i],sizes[i-1])
            parameters['b'+str(i)] = np.random.randn(sizes[i],1)
    return parameters

In [196]:
def measure_performance(X,Y, X_val, Y_val, params, activation_func, num_hidden_layers, loss):
    #we got final parameters
    #find Y_pred
    Yhat= np.zeros((Y.shape[0],Y.shape[1]))
    for j in range(0,X.shape[0]):
        x=X[j,:]
        y,_,_=fwd_prop(x,params,activation_func, num_hidden_layers)
        #print(y.shape)
        y=y.reshape(25,)
        Yhat[j,:]=y
    
        
    #Yhat, _, _ = fwd_prop(X, params, activation_func, num_hidden_layers)
    
    #print(Yhat.shape)
    #print(Y.shape)
    train_acc = find_accuracy(Yhat, Y)
    train_err = 100 - train_acc
    loss = loss_class(loss)
    train_loss = loss.calc_loss(Yhat, Y)
    train_loss = train_loss/float(X.shape[0])
    Yhat_val=np.zeros((Y_val.shape[0],Y_val.shape[1]))
    for j in range(0,X_val.shape[0]):
        x_val=X_val[j,:]
        y_val,_,_=fwd_prop(x,params, activation_func, num_hidden_layers)
        y_val=y_val.reshape(25,)
        Yhat_val[j,:] = y_val
    #print(Yhat_val)
    val_acc = find_accuracy(Yhat_val, Y_val)
    
    val_err = 100 - val_acc
    val_loss = loss.calc_loss(Yhat_val,Y_val)
    n=X_val.shape[0]
    #print(val_loss)
    val_loss=val_loss/float(n)
    '''print(train_err)
    print(train_loss)
    print(val_err)
    print(val_loss)'''
    return train_err, train_loss, val_err, val_loss
    

In [197]:
def find_accuracy(yhat,y):
    a = np.argmax(yhat, axis =0)
    
    b = np.argmax(y, axis =0)
    
    return 100*(np.sum( a == b)/len(a))

In [198]:
def plotfigure(xlabel,ylabel,title, x, y=[], figsize=(10,10), style = "k-", graph = "plot"):
  plt.figure(figsize = figsize)
  plt.grid(True)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.title(title)

  if(len(y) == 0):
    plt.plot(x,style)

  else:
    if graph == "plot":
      plt.plot(x,y,style)
    if graph == "semilogx":
      plt.semilogx(x,y,style)
    if graph == "semilogy":
      plt.semilogy(x,y,style)
    if graph == "loglog":
      plt.loglog(x,y,style)

    plt.tight_layout()
    #plt.show()
    plt.close()

In [199]:
def gen_sample_indices(batch_size,n):
  list_of_numbers = []
  for i in range(0,batch_size+1):
    list_of_numbers.append(random.randint(0, n))

  return list_of_numbers

In [200]:
def read_params(file_name):
    file = open(file_name ,"r")
    lines = file.readlines()
    hyper_parameters={}
    for i,line in enumerate(lines):
        value = line.strip()
        value = value.split()
        hyper_parameters[value[0]]=value[1]
    return hyper_parameters

In [201]:
def read_data(path_to_train, path_to_test):
    data = pd.read_csv(path_to_train)
    X=data.iloc[:,1:]
    X=X/255 #normalizing the pixel values   num samples x 784
    indices = data.iloc[:,0]
    Y = (convert_to_onehot(indices,25)) #num samples x 25
    pca = PCA(n_components=50) 
    pca.fit(X)
    #splitting the data to train and validation data
    
    X_train,X_val,y_train,y_val = train_test_split(X,Y,test_size = 0.2, random_state = 42)
    
    X_train = pca.transform(X_train)
    
    X_val = pca.transform(X_val)
    
    data_test = pd.read_csv(path_to_test)
    X_test=data_test.iloc[:,1:]
    X_test = X_test/255
    X_test = pca.transform(X_test)
    indices_test = data.iloc[:,0]
    y_test = convert_to_onehot(indices,25)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [202]:
def convert_to_onehot(indices, num_classes):

  output = np.eye(num_classes)[np.array(indices).reshape(-1)]

  return output.reshape(list(np.shape(indices))+[num_classes])

In [203]:
def createmomenta(num_hidden, sizes, inputsize, outputsize):
    sizes = [inputsize] + sizes
    sizes = sizes + [outputsize]
    momenta = {}
    for i in range(1, num_hidden+2):
        momenta["vW" + str(i)] = np.zeros((sizes[i], sizes[i-1]))
        momenta["vb" + str(i)] = np.zeros((sizes[i],1))
        # TODO: (5) scale these by 0.01 like in andrew ng's course?

    return momenta

In [204]:
def createmomenta_squared(num_hidden, sizes, inputsize, outputsize):
    sizes = [inputsize] + sizes
    sizes = sizes + [outputsize]
    momenta = {}
    for i in range(1, num_hidden+2):
        momenta["mW" + str(i)] = np.zeros((sizes[i], sizes[i-1]))
        momenta["mb" + str(i)] = np.zeros((sizes[i],1))
        # TODO: (5) scale these by 0.01 like in andrew ng's course?

    return momenta